In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn import cluster

    We're back again with 29,752 Jeopardy! questions and answers. Based on the text, we will compare two Unsupervised Learning topic generation models: Latent Dirichlet Allocation (LDA) and Non-Negative Matrix Factorization (NMF). These methods will cluster our questions into ten different topics. Then, we will use these generated features to create a Supervised regression model to predict if a question belongs to a certain topic. 

In [2]:
jeopardy = pd.read_csv('JEOPARDY_CSV.csv')

In [3]:
jeopardy.head()

,Show Number,Air Date,Round,Category,Value,Question,Answer
0,4680,2004-12-31,Jeopardy!,HISTORY,$200,"For the last 8 years of his life, Galileo was ...",Copernicus
1,4680,2004-12-31,Jeopardy!,ESPN's TOP 10 ALL-TIME ATHLETES,$200,No. 2: 1912 Olympian; football star at Carlisl...,Jim Thorpe
2,4680,2004-12-31,Jeopardy!,EVERYBODY TALKS ABOUT IT...,$200,The city of Yuma in this state has a record av...,Arizona
3,4680,2004-12-31,Jeopardy!,THE COMPANY LINE,$200,"In 1963, live on ""The Art Linkletter Show"", th...",McDonald's
4,4680,2004-12-31,Jeopardy!,EPITAPHS & TRIBUTES,$200,"Signer of the Dec. of Indep., framer of the Co...",John Adams


In [4]:
jeopardy.columns

Index(['Show Number', ' Air Date', ' Round', ' Category', ' Value',
       ' Question', ' Answer'],
      dtype='object')

In [5]:
#jeopardy.rename(columns={' Category':'category',
      # ' Question':'question', ' Answer':'answer'})

In [6]:
df = jeopardy[' Category'].value_counts().reset_index()

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29752 entries, 0 to 216929
Data columns (total 10 columns):
Show Number        29752 non-null int64
 Air Date          29752 non-null object
 Round             29752 non-null object
 Category          29752 non-null object
 Value             29752 non-null object
 Question          29752 non-null object
 Answer            29752 non-null object
question_answer    29752 non-null object
clean_qa           29752 non-null object
topics             29752 non-null int64
dtypes: int64(2), object(8)
memory usage: 3.7+ MB


In [8]:
df.columns = ['Category','count']

In [9]:
categories = list(df[df['count'] >= 100].Category)

In [10]:
type(categories)

list

In [11]:
categories

['BEFORE & AFTER',
 'SCIENCE',
 'LITERATURE',
 'AMERICAN HISTORY',
 'POTPOURRI',
 'WORLD HISTORY',
 'WORD ORIGINS',
 'COLLEGES & UNIVERSITIES',
 'HISTORY',
 'SPORTS',
 'U.S. CITIES',
 'WORLD GEOGRAPHY',
 'BODIES OF WATER',
 'ANIMALS',
 'STATE CAPITALS',
 'BUSINESS & INDUSTRY',
 'ISLANDS',
 'WORLD CAPITALS',
 'U.S. GEOGRAPHY',
 'RELIGION',
 'SHAKESPEARE',
 'OPERA',
 'LANGUAGES',
 'BALLET',
 'TELEVISION',
 'FICTIONAL CHARACTERS',
 'TRANSPORTATION',
 'PEOPLE',
 'RHYME TIME',
 'ART & ARTISTS',
 'STUPID ANSWERS',
 'THE BIBLE',
 'ART',
 'BOOKS & AUTHORS',
 'U.S. HISTORY',
 'FOOD',
 'MUSEUMS',
 'AMERICANA',
 'SCIENCE & NATURE',
 'COMMON BONDS',
 'HOLIDAYS & OBSERVANCES',
 'GEOGRAPHY',
 '3-LETTER WORDS',
 'ANNUAL EVENTS',
 'AMERICAN LITERATURE',
 'CLASSICAL MUSIC',
 'POP MUSIC',
 'AUTHORS',
 'POETS & POETRY',
 'QUOTATIONS',
 'HODGEPODGE',
 'MYTHOLOGY',
 'NONFICTION',
 'THE MOVIES',
 'WORLD CITIES',
 'MUSICAL INSTRUMENTS',
 'AROUND THE WORLD',
 'THE CIVIL WAR',
 'MUSIC',
 'U.S. PRESIDENTS',
 'C

In [12]:
df = jeopardy[jeopardy[' Category'].isin(categories)]

In [13]:
df['question_answer'] = df[' Question'] + df[' Answer']

C:\Users\spenser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

In [15]:
df['clean_qa'] = df['question_answer'].apply(lambda x: text_cleaner(x))

C:\Users\spenser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [16]:
import gensim
from gensim.models import word2vec

w2v = word2vec.Word2Vec(
    df['clean_qa'],
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

C:\Users\spenser\Anaconda3\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


done!


In [17]:
# List of words in model.
vocab = w2v.wv.vocab.keys()
print(vocab)
#print(w2v.wv.most_similar(positive=['army', 'man'], negative=['woman']))



dict_keys(['F', 'o', 'r', ' ', 't', 'h', 'e', 'l', 'a', 's', '8', 'y', 'f', 'i', ',', 'G', 'w', 'u', 'n', 'd', 'p', 'g', 'm', "'", 'C', 'c', 'I', 'A', 'b', 'B', '3', '1', '2', '.', 'k', 'R', '&', 'S', 'W', '0', 'j', 'O', '(', 'L', ')', 'v', '!', 'K', 'M', 'x', 'z', ';', '9', 'T', '5', '"', 'N', 'P', '4', '-', 'D', 'J', '6', '7', ':', 'q', 'E', 'H', 'V', 'U', '%', '<', '=', '/', '_', '>', 'Y', 'Z', '$', 'Q', '\\', '?', '“', '”', '’', 'X', 'é', 'í', 'á', 'ó', '#'])


In [18]:
#Xw2v = w2v.fit(df['clean_qa'])

In [19]:
tfidf = TfidfVectorizer(stop_words = 'english', max_df = 100, min_df = 10)

In [20]:
tfidf = tfidf.fit(df['clean_qa'])

In [21]:
X = tfidf.transform(df['clean_qa']) 

In [22]:
X

<29752x4206 sparse matrix of type '<class 'numpy.float64'>'
	with 107986 stored elements in Compressed Sparse Row format>

In [23]:
X = X.toarray()

In [24]:
X.shape

(29752, 4206)

In [25]:
lda = LatentDirichletAllocation(10, max_iter=10,
learning_method='online',
learning_offset=50.,
learning_decay = 0.7,
random_state=0)

In [26]:
lda = lda.fit(X)

In [27]:
X_lda = lda.transform(X)

In [28]:
def print_topics(model, vectorizer, top_n=10):
   for idx, topic in enumerate(model.components_):
       print("Topic %d:" % (idx))
       print([(vectorizer.get_feature_names()[i], topic[i])
                       for i in topic.argsort()[:-top_n - 1:-1]])
       print("=" * 100)

In [29]:
print("LDA Model:")
print_topics(lda, tfidf)

LDA Model:
Topic 0:
[('derived', 48.765983969795826), ('late', 43.748010347653953), ('using', 40.690556329113754), ('40', 39.615694081092002), ('gas', 36.839162477405011), ('wild', 36.264318116004894), ('jersey', 36.127885122826811), ('free', 34.78341105323252), ('divided', 34.620428959619915), ('metal', 34.413687974552097)]
Topic 1:
[('spoken', 56.795920474411581), ('dutch', 51.802380804478403), ('gulf', 46.475726256528212), ('street', 43.400453628951595), ('formed', 42.117864013156257), ('dog', 41.864878320078965), ('ship', 40.005207903012135), ('mark', 39.674446158105553), ('coast', 39.247481130956594), ('moon', 37.459316776122407)]
Topic 2:
[('takes', 46.621455684590238), ('member', 42.628964466279356), ('cities', 41.780903544413142), ('foot', 40.936432620922091), ('elected', 38.982990689831851), ('2009', 37.631029019798056), ('jefferson', 36.76631356468463), ('painting', 36.760347706155599), ('sir', 36.350601145399764), ('japanese', 36.146023581777271)]
Topic 3:
[('peak', 48.55908

As seen above, the topics generated by LDA mostly lack coherent meaning. 
    Topic 0: Misc.
    Topic 1: Misc. - Compare use of Dutch to NMF Topic 4
    Topic 2: Misc. - Politics, contains Jefferson and Japanese
    Topic 3: Some Geographical content
    Topic 4: Misc. - Contains mother and female, but also mount and Roosevelt
    Topic 5: Misc. - Maybe Shakespeare related
    Topic 6: Misc.
    Topic 7: Misc.
    Topic 8: Some Historical content
    Topic 9: Misc.

In [30]:
nmf = NMF(n_components=10, random_state=1,
          alpha=.1, l1_ratio=.5)

In [31]:
nmf.fit(X)

NMF(alpha=0.1, beta=1, eta=0.1, init=None, l1_ratio=0.5, max_iter=200,
  n_components=10, nls_max_iter=2000, random_state=1, shuffle=False,
  solver='cd', sparseness=None, tol=0.0001, verbose=0)

In [32]:
print_topics(nmf, tfidf)


Topic 0:
[('horse', 3.2482111937393352), ('race', 0.22403469654060248), ('racing', 0.16814883533728442), ('nile', 0.14291646486177598), ('crown', 0.13662653261471722), ('winged', 0.11750627784939911), ('triple', 0.11704734444098196), ('drawn', 0.088758738396920972), ('lee', 0.086461189776939396), ('wild', 0.084097009739033779)]
Topic 1:
[('spoken', 3.3403620019198379), ('languages', 0.92757637191391762), ('widely', 0.29467359115899316), ('dialect', 0.10853632261427593), ('united', 0.097903432557003608), ('morning', 0.080150318934918002), ('continentafrica', 0.064494877377648155), ('guinea', 0.049588259753491751), ('portuguese', 0.043083190231663453), ('continent', 0.040499163236133687)]
Topic 2:
[('peak', 2.582106579368824), ('mount', 2.2593353947847419), ('mountains', 0.98001854128311061), ('range', 0.67812349189283871), ('foot', 0.53927526981259366), ('everest', 0.28831995094196572), ('colorado', 0.26061232876312351), ('border', 0.17959748699966641), ('14', 0.16164233690501589), ('mt

    On the other hand, the NMF topics are much more closely clustered.
    Topic 0: Horse Racing
    Topic 1: Languages
    Topic 2: Mountains
    Topic 3: Misc. - Parts of speech and baseball
    Topic 4: Dutch Culture
    Topic 5: Oceanography
    Topic 6: Misc. - Some Science content
    Topic 7: African Culture / Geography
    Topic 8: Misc.
    Topic 9: Geography

In [33]:
nmf_X = nmf.transform(X)

In [34]:
nmf_X.shape

(29752, 10)

In [35]:
misc = []
topics = []
for i in range(nmf_X.shape[0]):
    
    topics.append(nmf_X[i].argsort()[-1])

In [36]:
df['topics'] = topics

C:\Users\spenser\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [37]:
nmf_X[4].argsort()[-1]

5

In [38]:

Y = df['topics']

In [40]:
df['topics'].value_counts()

9    27370
2      650
5      404
7      249
0      235
3      226
4      212
1      174
6      145
8       87
Name: topics, dtype: int64

In [53]:
df2 = df[df['topics'] = '9']


SyntaxError: invalid syntax (<ipython-input-53-81c728788183>, line 1)

In [59]:
df.loc['topics'] == 8

Show Number        True
 Air Date          True
 Round             True
 Category          True
 Value             True
 Question          True
 Answer            True
question_answer    True
clean_qa           True
topics             True
Name: topics, dtype: bool

In [41]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [42]:
log_regr = linear_model.LogisticRegression()
log_regr.fit(X_train, Y_train)

print('\nCoefficients: \n', log_regr.coef_)
print('\nIntercept: \n', log_regr.intercept_)
print('\nR-squared:')
print(log_regr.score(X_test, Y_test))
print(1 - (1-log_regr.score(X_test, Y_test))*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1))


Coefficients: 
 [[ 0.24578712 -0.10914346 -0.07565727 ..., -0.02225573 -0.0241235
  -0.03464927]
 [-0.05160611 -0.07120198 -0.05352192 ..., -0.01502025 -0.01349794
  -0.02217961]
 [-0.16187153 -0.22391032 -0.15898888 ..., -0.04613455 -0.04996332
  -0.07198022]
 ..., 
 [-0.0715992  -0.12556724 -0.06884514 ..., -0.02022884 -0.01986579
  -0.0320533 ]
 [-0.0287491  -0.04100943 -0.02843624 ..., -0.00812084 -0.00766585
  -0.0126783 ]
 [-0.19460234  0.09209627  0.3096135  ...,  0.12955731  0.19647281
  -0.23332211]]

Intercept: 
 [-5.26157525 -5.72637163 -4.51297475 -5.37844581 -5.46162696 -4.95333437
 -5.82559735 -5.34517442 -6.29654017  3.25681077]

R-squared:
0.954850997087
0.914610118458
